# GRHydro Tutorial

This tutorial is based on the [Einstein Toolkit tutorial](https://github.com/nds-org/jupyter-et/blob/master/tutorial-server/notebooks/CactusTutorial.ipynb) and it assumes that you have already installed the Einstein Toolkit.

It will require the installation of [kuibit](https://sbozzolo.github.io/kuibit/) in order to make some 2D plots and this is the first thing that will be done in the next cell.

In [ ]:
!pip install kuibit

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import kuibit.cactus_scalars as cs
import kuibit.simdir as sd

from kuibit import grid_data as gd

We now go into the Cactus directory (make sure to change the path accordingly to your system)

In [ ]:
cd ~/EinsteinToolkit/ET_2023_05/Cactus

In [ ]:
!pwd

We now create a parameter file to evolve a non-rotating stable neutron star with a polytropic equation of state

In [ ]:
%%bash
cat >par/TOV_ET_Bruno.par <<"#EOF"
# Some basic stuff
ActiveThorns = "Time MoL"
ActiveThorns = "Coordbase CartGrid3d Boundary StaticConformal"
ActiveThorns = "SymBase ADMBase TmunuBase HydroBase InitBase ADMCoupling ADMMacros"
ActiveThorns = "IOUtil"
ActiveThorns = "Formaline"
ActiveThorns = "SpaceMask CoordGauge Constants LocalReduce aeilocalinterp LoopControl"
ActiveThorns = "Carpet CarpetLib CarpetReduce CarpetRegrid2 CarpetInterp"
ActiveThorns = "CarpetIOASCII CarpetIOScalar CarpetIOHDF5 CarpetIOBasic"

# Finalize
Cactus::terminate           = "time"
Cactus::cctk_final_time     = 400 # divide by ~203 to get ms

# Termination Trigger
ActiveThorns = "TerminationTrigger"
TerminationTrigger::max_walltime = 24          # hours
TerminationTrigger::on_remaining_walltime = 0  # minutes
TerminationTrigger::check_file_every = 512
TerminationTrigger::termination_file = "TerminationTrigger.txt"
TerminationTrigger::termination_from_file   = "yes"
TerminationTrigger::create_termination_file = "yes"

# grid parameters
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type         = "coordbase"
CartGrid3D::domain       = "full"
CartGrid3D::avoid_origin = "no"
CoordBase::xmin =  0.0
CoordBase::ymin =  0.0
CoordBase::zmin =  0.0
CoordBase::xmax = 24.0
CoordBase::ymax = 24.0
CoordBase::zmax = 24.0
# Change these parameters to change resolution. The ?max settings above
# have to be multiples of these. 'dx' is the size of one cell in x-direction.
# Making this smaller means using higher resolution, because more points will
# be used to cover the same space.
CoordBase::dx   =   2.0
CoordBase::dy   =   2.0
CoordBase::dz   =   2.0

CarpetRegrid2::regrid_every =   0
CarpetRegrid2::num_centres  =   1
CarpetRegrid2::num_levels_1 =   2
CarpetRegrid2::radius_1[1]  = 12.0


CoordBase::boundary_size_x_lower        = 3
CoordBase::boundary_size_y_lower        = 3
CoordBase::boundary_size_z_lower        = 3
CoordBase::boundary_size_x_upper        = 3
CoordBase::boundary_size_y_upper        = 3
CoordBase::boundary_size_z_upper        = 3
CoordBase::boundary_shiftout_x_lower    = 1
CoordBase::boundary_shiftout_y_lower    = 1
CoordBase::boundary_shiftout_z_lower    = 1
CoordBase::boundary_shiftout_x_upper    = 0
CoordBase::boundary_shiftout_y_upper    = 0
CoordBase::boundary_shiftout_z_upper    = 0


ActiveThorns = "ReflectionSymmetry"

ReflectionSymmetry::reflection_x = "yes"
ReflectionSymmetry::reflection_y = "yes"
ReflectionSymmetry::reflection_z = "yes"
ReflectionSymmetry::avoid_origin_x = "no"
ReflectionSymmetry::avoid_origin_y = "no"
ReflectionSymmetry::avoid_origin_z = "no"

# storage and coupling
TmunuBase::stress_energy_storage = yes
TmunuBase::stress_energy_at_RHS  = yes
TmunuBase::timelevels            =  1
TmunuBase::prolongation_type     = none


HydroBase::timelevels            = 3

ADMMacros::spatial_order = 4

SpaceMask::use_mask      = "yes"

Carpet::enable_all_storage       = no
Carpet::use_buffer_zones         = "yes"

Carpet::poison_new_timelevels    = "yes"
Carpet::check_for_poison         = "no"

Carpet::init_3_timelevels        = no
Carpet::init_fill_timelevels     = "yes"

CarpetLib::poison_new_memory = "yes"
CarpetLib::poison_value      = 114

# system specific Carpet paramters
Carpet::max_refinement_levels    = 10
driver::ghost_size               = 3
Carpet::prolongation_order_space = 3
Carpet::prolongation_order_time  = 2

# Time integration
time::dtfac = 0.25

MoL::ODE_Method             = "rk4"
MoL::MoL_Intermediate_Steps = 4
MoL::MoL_Num_Scratch_Levels = 1

# check all physical variables for NaNs
#  This can save you computing time, so it's not a bad idea to do this
#  once in a whioe.
ActiveThorns = "NaNChecker"
NaNChecker::check_every = 16384
NaNChecker::action_if_found = "terminate" #"terminate", "just warn", "abort"
NaNChecker::check_vars = "ADMBase::metric ADMBase::lapse ADMBase::shift HydroBase::rho HydroBase::eps HydroBase::press HydroBase::vel"

# Hydro paramters

ActiveThorns = "EOS_Omni GRHydro"

HydroBase::evolution_method      = "GRHydro"

GRHydro::riemann_solver         = "Marquina"
GRHydro::GRHydro_eos_type       = "Polytype"
GRHydro::GRHydro_eos_table      = "2D_Polytrope"
GRHydro::recon_method           = "ppm"
GRHydro::GRHydro_stencil        = 3
GRHydro::bound                  = "none"
GRHydro::rho_abs_min            = 1.e-10
# Parameter controlling finite difference order of the Christoffel symbols in GRHydro
GRHydro::sources_spatial_order  = 4

# Curvature evolution parameters

ActiveThorns = "GenericFD NewRad"
ActiveThorns = "ML_BSSN ML_BSSN_Helper"
ADMBase::evolution_method        = "ML_BSSN"
ADMBase::lapse_evolution_method  = "ML_BSSN"
ADMBase::shift_evolution_method  = "ML_BSSN"
ADMBase::dtlapse_evolution_method= "ML_BSSN"
ADMBase::dtshift_evolution_method= "ML_BSSN"

ML_BSSN::timelevels = 3

ML_BSSN::harmonicN           = 1      # 1+log
ML_BSSN::harmonicF           = 2.0    # 1+log
ML_BSSN::evolveA             = 1
ML_BSSN::evolveB             = 1
ML_BSSN::ShiftGammaCoeff     = 0.75
ML_BSSN::BetaDriver          = 2.66
ML_BSSN::advectLapse         = 0.0
ML_BSSN::advectShift         = 0.0


ML_BSSN::initial_boundary_condition="extrapolate-gammas"
ML_BSSN::rhs_boundary_condition="NewRad"

# Some dissipation to get rid of high-frequency noise
ActiveThorns = "SphericalSurface Dissipation"
Dissipation::verbose   = "no"
Dissipation::epsdis   = 0.01
Dissipation::vars = "
        ML_BSSN::ML_log_confac
        ML_BSSN::ML_metric
        ML_BSSN::ML_curv
        ML_BSSN::ML_trace_curv
        ML_BSSN::ML_Gamma
        ML_BSSN::ML_lapse
        ML_BSSN::ML_shift
"


# init parameters
InitBase::initial_data_setup_method = "init_some_levels"

# Use TOV as initial data
ActiveThorns = "TOVSolver"

HydroBase::initial_hydro         = "tov"
ADMBase::initial_data            = "tov"
ADMBase::initial_lapse           = "tov"
ADMBase::initial_shift           = "tov"
ADMBase::initial_dtlapse         = "zero"
ADMBase::initial_dtshift         = "zero"

# Parameters for initial star
TOVSolver::TOV_Rho_Central[0] = 1.28e-3
TOVSolver::TOV_Gamma          = 2
TOVSolver::TOV_K              = 100

# Set equation of state for evolution
EOS_Omni::poly_gamma                   = 2
EOS_Omni::poly_k                       = 100
EOS_Omni::gl_gamma                     = 2
EOS_Omni::gl_k                         = 100


# I/O

# Use (create if necessary) an output directory named like the
# parameter file (minus the .par)
IO::out_dir             = ${parfile}

# Write one file overall per output (variable/group)
# In production runs, comment this or set to "proc" to get one file
# per MPI process
IO::out_mode            = "onefile"

# Some screen output
IOBasic::outInfo_every = 512
IOBasic::outInfo_vars  = "Carpet::physical_time_per_hour HydroBase::rho{reductions='maximum'}"

# Scalar output
IOScalar::outScalar_every    = 512
IOScalar::one_file_per_group = "yes"
IOScalar::outScalar_reductions = "norm1 norm2 norm_inf sum maximum minimum"
IOScalar::outScalar_vars     = "
 HydroBase::rho{reductions='maximum'}
 HydroBase::press{reductions='maximum'}
 HydroBase::eps{reductions='minimum maximum'}
 HydroBase::vel{reductions='minimum maximum'}
 HydroBase::w_lorentz{reductions='minimum maximum'}
 ADMBase::lapse{reductions='minimum maximum'}
 ADMBase::shift{reductions='minimum maximum'}
 ML_BSSN::ML_Ham{reductions='norm1 norm2 maximum minimum norm_inf'}
 ML_BSSN::ML_mom{reductions='norm1 norm2 maximum minimum norm_inf'}
 GRHydro::dens{reductions='minimum maximum sum'}
 Carpet::timing{reductions='average'}
"

# 1D ASCII output. Disable for production runs!
IOASCII::out1D_every        = -1
IOASCII::one_file_per_group = yes
IOASCII::output_symmetry_points = no
IOASCII::out1D_vars         = "
 HydroBase::rho
 HydroBase::press
 HydroBase::eps
 HydroBase::vel
 ADMBase::lapse
 ADMBase::metric
 ADMBase::curv
 ML_BSSN::ML_Ham
 ML_BSSN::ML_mom
"

# 2D HDF5 output
CarpetIOHDF5::output_buffer_points = "no"

CarpetIOHDF5::out2D_every = 2048
CarpetIOHDF5::out2D_vars = "
 HydroBase::rho
 HydroBase::eps
 HydroBase::vel
 HydroBase::w_lorentz
 ADMBase::lapse
 ADMBase::shift
 ADMBase::metric
 ML_BSSN::ML_Ham
 ML_BSSN::ML_mom
 "

#EOF

We now create and submit our simulation

In [ ]:
%%bash
# create simulation directory structure
./simfactory/bin/sim create TOV_ET_Bruno --configuration sim --parfile=par/TOV_ET_Bruno.par

In [ ]:
%%bash
# start simulation segment
./simfactory/bin/sim submit TOV_ET_Bruno --cores=4 --num-threads=4 --walltime=0:30:00

In [ ]:
%%bash
./simfactory/bin/sim list-simulations TOV_ET_Bruno

In [ ]:
%%bash
# watch log output, following along as new output is produced
./simfactory/bin/sim show-output TOV_ET_Bruno

We now check where the output of this run has been saved.

In [ ]:
%%bash
./simfactory/bin/sim get-output-dir TOV_ET_Bruno

In [ ]:
sim = sd.SimDir("/home/bgiacoma/simulations/TOV_ET_Bruno")

In [ ]:
timeseries = sim.timeseries
print(timeseries)

We read the maximum of the rest-mass density and plot it

In [ ]:
rho_max=timeseries.maximum.fields.rho

In [ ]:
plt.plot(rho_max, label="maximum rest-mass density")
plt.xlabel(r'$t$ [$M_{\odot}$]');
plt.ylabel(r'$\rho_c$');
plt.legend();

### 2D Plots with Kuibit

We now do a simple 2D plot of the rest-mass density using kuibit

In [ ]:
gf = sim.gf
print(gf)

In [ ]:
vars2D = gf.xy
print(vars2D)

In [ ]:
rho = vars2D.fields.rho

In [ ]:
print(rho.iterations)

In [ ]:
print(rho.available_times)

In [ ]:
small_grid = gd.UniformGrid([100, 100], x0=[0, 0], x1=[12,12])

rho_small = rho.read_on_grid(409600, small_grid)

cf = plt.pcolormesh(*rho_small.coordinates_meshgrid(), np.log10(rho_small.data_xyz), vmin=-10, vmax=-3)
plt.colorbar(cf)

The following cells may take some time and they can be used to produce an animation.

In [ ]:
#for i in rho.iterations:
#    print(i)
#    rho_small = rho.read_on_grid(i, small_grid)
#    cf2=plt.contourf(*rho_small.coordinates_meshgrid(), np.log10(rho_small.data_xyz), vmin=-10, vmax=-3)
#    plt.savefig("frame_"+str(i)+".png")

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
ims = []
fig,ax = plt.subplots()

for i in rho.iterations[0::1]:
    rho_small = rho.read_on_grid(i, small_grid)
    ax.set_xlabel(r'$x (M_\odot)$')
    ax.set_ylabel(r'$y (M_\odot)$')
    ax.set_aspect('equal')
    im = ax.pcolormesh(*rho_small.coordinates_meshgrid(), np.log10(rho_small.data_xyz), animated=True, vmin=-10, vmax=-3)
    plt.plot()
    plt.colorbar(im)
    title = ax.text(0.5,1.05,('Time='+str(rho.available_times[i//2048])), size=plt.rcParams["axes.titlesize"], ha="center", transform=ax.transAxes)
    ims.append([im, title])
    plt.close()
    

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
HTML(ani.to_jshtml())

### Binary Neutron Stars

Here we will read an initial data file produced with [LORENE](https://lorene.obspm.fr/) and representing an equal-mass neutron star binary system. We will only save the initial data and visualize it, since an actual simulation will require an higher resolution and it may not run on a laptop.

We use the same initial data used for the [Einstein Toolkit gallery example](http://einsteintoolkit.org/gallery/bns/index.html)

In [ ]:
cd /home/bgiacoma/simulations/

In [ ]:
!ls
!pwd

In [ ]:
!wget http://einsteintoolkit.org/gallery/bns/G2_I12vs12_D4R33T21_45km.resu.xz
!unxz G2_I12vs12_D4R33T21_45km.resu.xz
!ls
!pwd

We now create the parameter file

In [ ]:
cd ~/EinsteinToolkit/ET_2023_05/Cactus

In [ ]:
!pwd

In [ ]:
%%bash
cat >par/BNS_ET_Bruno.par <<"#EOF"
#------------------------------------------------------------------------------
# Cactus parameters:
#------------------------------------------------------------------------------
Cactus::cctk_run_title     = "Meudon BNS"
Cactus::cctk_full_warnings = "yes"
Cactus::highlight_warning_messages = "no"

Cactus::terminate       = "time"
Cactus::cctk_final_time = 0.0 #only initial data

#------------------------------------------------------------------------------
# Activate all necessary thorns:
#------------------------------------------------------------------------------

ActiveThorns = "Boundary CartGrid3D CoordBase Fortran InitBase IOUtil LocalReduce SymBase Time"
ActiveThorns = "AEILocalInterp"
ActiveThorns = "MoL Slab SpaceMask SphericalSurface"
ActiveThorns = "Carpet CarpetInterp CarpetInterp2 CarpetIOASCII CarpetIOHDF5 CarpetIOScalar CarpetLib CarpetIOBasic CarpetReduce CarpetRegrid2 CarpetSlab LoopControl"
ActiveThorns = "Formaline"
ActiveThorns = "NaNChecker"
ActiveThorns = "ADMbase ADMcoupling ADMmacros CoordGauge StaticConformal"
ActiveThorns = "ReflectionSymmetry"
ActiveThorns = "Constants TmunuBase HydroBase "
ActiveThorns = "QuasiLocalMeasures"
ActiveThorns = "EOS_Omni"
ActiveThorns = "GRHydro"
ActiveThorns = "SummationByParts"
ActiveThorns = "GenericFD NewRad"
ActiveThorns = "ML_BSSN ML_BSSN_Helper ML_ADMConstraints"
ActiveThorns = "Hydro_Analysis"
ActiveThorns = "Dissipation"
ActiveThorns = "SystemStatistics SystemTopology"

#------------------------------------------------------------------------------
# Utility parameters:
#------------------------------------------------------------------------------

NaNChecker::check_every    =  128 # twice for every_coarse
NaNChecker::check_vars = "
            ADMBase::curv 
            ADMBase::metric 
            ADMBase::lapse 
            ADMBase::shift 
            HydroBase::rho 
            HydroBase::eps 
            HydroBase::press 
            HydroBase::vel
"
NaNChecker::action_if_found   =  "terminate"
#NaNChecker::action_if_found = "just warn" #"terminate", "just warn", "abort"

#------------------------------------------------------------------------------
# Run parameters:
#------------------------------------------------------------------------------

#------
# Grid:
#------

MoL::ODE_Method             = "rk4"
MoL::MoL_Intermediate_Steps = 4
MoL::MoL_Num_Scratch_Levels = 1
# use dt = 0.4 dx (works for core collapse)
Time::dtfac = 0.4



ActiveThorns = "CoordBase"

CoordBase::domainsize = "minmax"

CoordBase::xmin = -100.00
CoordBase::ymin = -100.00
CoordBase::zmin =    0.00
CoordBase::xmax = +100.00
CoordBase::ymax = +100.00
CoordBase::zmax = +100.00
CoordBase::dx   =    10.00
CoordBase::dy   =    10.00
CoordBase::dz   =    10.00

CoordBase::boundary_size_x_lower     = 3
CoordBase::boundary_size_y_lower     = 3
CoordBase::boundary_size_z_lower     = 3
CoordBase::boundary_size_x_upper     = 3
CoordBase::boundary_size_y_upper     = 3
CoordBase::boundary_size_z_upper     = 3

CoordBase::boundary_shiftout_x_lower = 0
CoordBase::boundary_shiftout_y_lower = 0
CoordBase::boundary_shiftout_z_lower = 1

ReflectionSymmetry::reflection_z   = "yes"
ReflectionSymmetry::avoid_origin_z = "no"

CartGrid3D::type = "coordbase"
Carpet::domain_from_coordbase = "yes"

Driver::ghost_size                      = 3


# General Carpet parameters:
Carpet::enable_all_storage       = "no"
Carpet::use_buffer_zones         = "yes"

Carpet::init_3_timelevels        = "no"
Carpet::init_fill_timelevels     = "yes"

CarpetRegrid2::snap_to_coarse          = "yes"

# System specific Carpet parameters:
Carpet::max_refinement_levels    = 10 
Carpet::prolongation_order_space = 5
Carpet::prolongation_order_time  = 2

CarpetRegrid2::regrid_every = 0 
CarpetRegrid2::num_centres  = 1

CarpetRegrid2::num_levels_1 = 3
CarpetRegrid2::radius_1[1]  =50.0
CarpetRegrid2::radius_1[2]  =30.0


#------
# MODEL:
#------

ActiveThorns = "Meudon_Bin_NS"
HydroBase::initial_hydro         = "Meudon_Bin_NS"
ADMBase::initial_data            = "Meudon_Bin_NS"
ADMBase::initial_lapse           = "Meudon_Bin_NS"
ADMBase::initial_shift           = "zero"
ADMBase::initial_dtlapse         = "Meudon_Bin_NS"
ADMBase::initial_dtshift         = "zero"

# change this to be the full path to he initial data file
Meudon_Bin_NS::filename ="/home/bgiacoma/simulations/G2_I12vs12_D4R33T21_45km.resu"

EOS_Omni::poly_K = 123.613314525753

#----------
# Numerics:
#----------

InitBase::initial_data_setup_method = "init_some_levels"

TmunuBase::stress_energy_storage = "yes"
TmunuBase::stress_energy_at_RHS  = "yes"
TmunuBase::timelevels            =  1
TmunuBase::prolongation_type     = "none"

HydroBase::timelevels            = 3

SpaceMask::use_mask      = "yes"


#-----------
# Evolution:
#-----------

HydroBase::evolution_method      = "GRHydro"

ADMMacros::spatial_order = 4
GRHydro::sources_spatial_order = 4

GRHydro::riemann_solver            = "HLLE"   # Marquina is currently not supported by MP
GRHydro::recon_method              = "ppm"
GRHydro::use_enhanced_ppm            = "yes"
GRHydro::GRHydro_stencil            = 3
GRHydro::bound                     = "flat"
GRHydro::rho_abs_min               = 1.e-11
GRHydro::GRHydro_atmo_tolerance    = 0.01

GRHydro::c2p_reset_pressure        = "yes"

GRHydro::GRHydro_eos_type           = "General"
GRHydro::GRHydro_eos_table          = "Ideal_Fluid"

# these can save some memory since they prevent MoL from allocating unnecessary
# scratch space for saveandrestore variables
GRHydro::GRHydro_MaxNumSandRVars = 0

GRHydro::sync_conserved_only     = "yes"
GRHydro::reconstruct_Wv          = "yes"
GRHydro::c2p_resort_to_bisection = "yes"
GRHydro::use_cxx_code            = "yes"


# MacLachlan evolution parameters

ADMBase::metric_type                    = physical
ADMBase::evolution_method               = ML_BSSN
ADMBase::lapse_evolution_method         = ML_BSSN
ADMBase::shift_evolution_method         = ML_BSSN
ADMBase::dtlapse_evolution_method       = ML_BSSN
ADMBase::dtshift_evolution_method       = ML_BSSN


ML_BSSN::timelevels                     = 3
ML_BSSN::initial_boundary_condition     = "extrapolate-gammas"
ML_BSSN::rhs_boundary_condition         = "NewRad"
Boundary::radpower                      = 2 # not really needed I think but who knows what NewRad uses

ML_BSSN::harmonicN           = 1      # 1+log
ML_BSSN::harmonicF           = 2.0    # 1+log
ML_BSSN::ShiftGammaCoeff     = 0.75
ML_BSSN::AlphaDriver         = 0.0
ML_BSSN::BetaDriver          = 1.0
ML_BSSN::advectLapse         = 1.0
ML_BSSN::advectShift         = 1.0

ML_BSSN::MinimumLapse = 1.0e-8
ML_BSSN::ML_log_confac_bound = "none"
ML_BSSN::ML_metric_bound     = "none"
ML_BSSN::ML_Gamma_bound      = "none"
ML_BSSN::ML_trace_curv_bound = "none"
ML_BSSN::ML_curv_bound       = "none"
ML_BSSN::ML_lapse_bound      = "none"
ML_BSSN::ML_dtlapse_bound    = "none"
ML_BSSN::ML_shift_bound      = "none"
ML_BSSN::ML_dtshift_bound    = "none"

Dissipation::epsdis = 0.1
Dissipation::order = 5
Dissipation::vars                       = "
        ML_BSSN::ML_log_confac
        ML_BSSN::ML_metric
        ML_BSSN::ML_trace_curv
        ML_BSSN::ML_curv
        ML_BSSN::ML_Gamma
        ML_BSSN::ML_lapse
        ML_BSSN::ML_shift
        ML_BSSN::ML_dtlapse
        ML_BSSN::ML_dtshift
"

#------------------------------------------------------------------------------
# Output:
#------------------------------------------------------------------------------

IO::out_dir = $parfile

IOBasic::outInfo_every = 1
IOBasic::outInfo_reductions = "maximum"
IOBasic::outInfo_vars  = "
 Carpet::physical_time_per_hour
 HydroBase::rho
 ML_ADMConstraints::ML_Ham
"

IOScalar::outScalar_every      = 1
IOScalar::all_reductions_in_one_file = "yes"
IOScalar::one_file_per_group   = "yes"
IOScalar::outScalar_reductions = "minimum maximum average norm1 norm2"
IOScalar::outScalar_vars       = "
 ADMBase::lapse
 ADMBase::shift
 ADMBase::metric
 ADMBase::curv
 HydroBase::rho
 HydroBase::vel
 HydroBase::w_lorentz
 GRHydro::dens{reductions = 'minimum maximum average norm1 norm2 sum'}
 ML_ADMConstraints::ML_Ham
"

IOASCII::one_file_per_group     = "yes"
IOASCII::compact_format  = "yes"

IOASCII::out0D_every     = 1
IOASCII::out0D_vars      = "
 Carpet::timing
"

CarpetIOHDF5::one_file_per_group             = "no" 
CarpetIOHDF5::open_one_input_file_at_a_time  = "yes"
CarpetIOHDF5::out2D_every                    = 1
CarpetIOHDF5::out2D_vars      = "
  HydroBase::rho
  HydroBase::vel
  HydroBase::eps
  ADMBase::lapse
  ADMBase::shift
  ADMBase::metric
  ML_ADMConstraints::ML_Ham
 "

IOHDF5::out3D_every = 1 
IOHDF5::out3D_vars  = "
 HydroBase::rho
 ADMBase::lapse
"

#------------------------------------------------------------------------------
# Analysis:
#------------------------------------------------------------------------------

#EOF

In [ ]:
%%bash
# create simulation directory structure
./simfactory/bin/sim create BNS_ET_Bruno --configuration sim --parfile=par/BNS_ET_Bruno.par

In [ ]:
%%bash
# start simulation segment
./simfactory/bin/sim submit BNS_ET_Bruno --cores=4 --num-threads=4 --walltime=0:30:00

In [ ]:
%%bash
./simfactory/bin/sim list-simulations BNS_ET_Bruno

In [ ]:
%%bash
# watch log output, following along as new output is produced
./simfactory/bin/sim show-output BNS_ET_Bruno

In [ ]:
%%bash
./simfactory/bin/sim get-output-dir BNS_ET_Bruno

In [ ]:
sim_bns = sd.SimDir("/home/bgiacoma/simulations/BNS_ET_Bruno")

In [ ]:
gf_bns = sim_bns.gf
print(gf_bns)

In [ ]:
rho_bns = gf_bns.xy.fields.rho
alp_bns = gf_bns.xy.fields.alp
H_bns = gf_bns.xy.fields.H

In [ ]:
small_grid = gd.UniformGrid([200, 200], x0=[-30, -30], x1=[30,30])

rho_small = rho_bns.read_on_grid(0, small_grid)

cf = plt.pcolormesh(*rho_small.coordinates_meshgrid(), np.log10(rho_small.data_xyz))
plt.colorbar(cf)

In [ ]:
small_grid = gd.UniformGrid([200, 200], x0=[-90, -90], x1=[90,90])
alp_small = alp_bns.read_on_grid(0, small_grid)

cf = plt.pcolormesh(*alp_small.coordinates_meshgrid(), alp_small.data_xyz)
plt.colorbar(cf)

In [ ]:
H_small = H_bns.read_on_grid(0, small_grid)

cf = plt.pcolormesh(*H_small.coordinates_meshgrid(), np.log10(abs(H_small.data_xyz)))
plt.colorbar(cf)